## Introduction

This chapter addresses three major topics that are closely related:
- Python's `async def`, `await`, `async with`, and `async for` constructs
- Objects supporting those constructs: native coroutines and asynchronous variants of context managers, iterables, generators, and comprehensions
- `asyncio` and other asynchronous libraries

## A Few Definitions
- <span style="color:skyblue">***Native coroutine***</span>: A coroutine function defined with `async def`. You can delegate from a native coroutine to another native coroutine using the `await` keyword, similar to how classic coroutines use `yield` from. The `async def` statement always defines a native coroutine, even if the await keyword is not used in its body. The `await` keyword cannot be used outside of a native coroutine
- <span style="color:skyblue">***Classic coroutine***</span>: A generator function that consumes data sent to it via `my_coro.send(data)` calls, and reads that data by using `yield` in an expression. Classic coroutines can delegate to other classic coroutines using `yield` from. Classic coroutines cannot be driven by `await`, and are no longer supported by `asyncio`.
- <span style="color:skyblue">***Generator-based coroutine***</span>: A generator function decorated with `@types.coroutine` — introduced in Python 3.5. That decorator makes the generator compatible with the new `await` keyword.
- <span style="color:skyblue">***Asynchronous generator***</span>: A generator function defined with `async def` and using `yield` in its body. It returns an asynchronous generator object that provides `__anext__`, a coroutine method to retrieve the next item.

## An `asyncio` Example: Probing Domains

We want to write a script (`blogdom.py`) to concurrently check if domains `<name>.dev` are available for our Python blog. The output should be  
```bash
python3 blogdom.py
with.dev
+ elif.dev
+ def.dev
from.dev
```
Compared to checking domains sequentially, probing DNS via native coroutine objects should be much faster, almost the same as the time to check the single slowest DNS, instead of the sum of all responses

In [ ]:
import asyncio
import socket
from keyword import kwlist

MAX_KEYWORD_LEN = 4  # Max length of keyword for domains, because shorter is better.


async def probe(domain: str) -> tuple[str, bool]:  # <2>
    loop = asyncio.get_running_loop()  # <3>
    try:
        await loop.getaddrinfo(domain, None)  # <4>
    except socket.gaierror:
        return (domain, False)
    return (domain, True)


async def main() -> None:  # <5>
    names = (kw for kw in kwlist if len(kw) <= MAX_KEYWORD_LEN)  # <6>
    domains = (f'{name}.dev'.lower() for name in names)  # <7>
    coros = [probe(domain) for domain in domains]  # <8>
    for coro in asyncio.as_completed(coros):  # <9>
        domain, found = await coro  # <10>
        mark = '+' if found else ' '
        print(f'{mark} {domain}')